In [ ]:
import numpy as np

ap = argparse.ArgumentParser()
args = {
        "config": './yolov3-tiny.cfg',
        "weights": './yolov3-tiny_best_1.weights',
        "classes": './object.names'
        }
#args = ap.parse_args()


# Get names of output layers, output for YOLOv3 is ['yolo_16', 'yolo_23']
def getOutputsNames(net):
    layersNames = net.getLayerNames()
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]


# Darw a rectangle surrounding the object and its class name 
def draw_pred(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])
    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


# Double Engine

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Feb  7 12:43:04 2020

@author: IN0193
"""

import cv2
import numpy as np
import time
import datetime
from connector_api import cameraConnector
from classify import classify

class CaptureRate:
    
    def __init__(self, path_config_file, path_weights_file, path_labels_file, video_source = 0, 
                 cap_rate = 4, patience = 30, thresh = 2):
        
        self.path_config_file = path_config_file
        self.path_weights_file = path_weights_file
        self.path_labels_file = path_labels_file
        self.video_source = video_source                # Default is for the webcam
        self.cap_rate = cap_rate                        # Number of frames per second to be captured
        
        # Patience and Thresholding
        self.patience = patience
        self.thresh = thresh
        
        # Initial state of flags 
        self.flag_patience = True
        self.flag_thresh = True
        
    def load_network(self):
        # Read YOLO cfg file and weights file
        net = cv2.dnn.readNetFromDarknet(self.path_config_file, self.path_weights_file)
        print('[INFO] Network is now loaded...')
        return (net)
    
    
    def load_classes(self):
                
        # List of classes from the text file
        classes = open(self.path_labels_file).read().strip().split('\n')
        print('[INFO] Classes initialized...')
        return classes
    
    
    def connector_api(self):
        
        msg = cameraConnector()
        print('[INFO] Camera Loaded...')
        cap , fps = msg.directConnector()
        return cap
        
             
    def main_run(self):
        net = self.load_network()
        classes = self.load_classes()
         
        # Get the layers names and select the output layers out of them
        layer_names = net.getLayerNames()
        outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
        
        # Initialize the colours of the bounding boxes
        colors= np.random.uniform(0,255,size=(len(classes),3))
        
        #loading capturing frames from webcam
        
        
        #########################################
        #Getting the frame per second of the feed
        #########################################
        # self.frame_rate = cap.get(cv2.CAP_PROP_FPS)
        
        font = cv2.FONT_HERSHEY_PLAIN
        # starting_time= time.time()
        # frame_id = 0
        
        frame_counter = 1        
        cap = self.connector_api()
        self.frame_rate = 20
        while True:
            
            
            
            print('Frame ID: {}'.format(frame_counter))
            print('[INFO] Entered Loop..')
            grabbed ,frame = cap.read()
            # cv2.imshow('frame',frame)
            
            # if cv2.waitKey(1) == ord('q'):
            #     cap.release()
            #     cv2.destroyAllWindows()
            #     break
            ###########################################################
            # Set the condition for skipping frames from the video feed
            ###########################################################
            
            condition = int(self.frame_rate // self.cap_rate)
            if (frame_counter == condition):
                
                # grabbed ,frame = cap.read()
                    
                if grabbed:
                    print('[INFO] New Frame Grabbed...')
            
                # Get frame height, width, channels
                height,width,channels = frame.shape
                
                # Blob from the image
                blob = cv2.dnn.blobFromImage(frame,0.00392,(416,416),(0,0,0),True,crop=False)
                
                # Set input to the network
                net.setInput(blob)
                
                # Set the input through all the output layers and get output
                outs = net.forward(outputlayers)
                
                class_ids=[]
                confidences=[]
                boxes=[]
                
                for out in outs:
                    for detection in out:
                        scores = detection[5:]
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        
                        # Detect the objects with confidences more than mentioned input
                        if confidence > 0.3:
                            
                            center_x= int(detection[0]*width)
                            center_y= int(detection[1]*height)                            
                            w = int(detection[2]*width)
                            h = int(detection[3]*height)
                            
                            x=int(center_x - w/2)
                            y=int(center_y - h/2)
                            
                            boxes.append([x,y,w,h]) #put all rectangle areas
                            confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
                            class_ids.append(class_id) #name of the object tha was detected
                
                # Non Max Supression of the detected objects, with input as confidence score and 
                # threshold values
                indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.65 ,0.4)
                
                for i in range(len(boxes)):
                    if i in indexes:
                        x,y,w,h = boxes[i]
                        label = str(classes[class_ids[i]])
                        if label == "person" and x>0 and y>0 and h>80 and w>20:
                            image = frame[y:y+h, x:x+w]
                            print(image.shape, y,y+h, x,w+x)
                            x1, y1, w1, h1, label, confidence = classify(image)
                            print( x,y,w,h, label,confidence)
#                             confidence= confidences[i]
#                             color = colors[class_ids[i]]
                            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,255),1)
                            cv2.putText(frame,str(label) +" "+str(round(confidence,2)),(x,y+30),font,1,(255,255,255),2)
                        ##################################################
                        #Condition for setting up PATIENCE and THRSHOLDING   
                        ##################################################
                        
#                         if (label == 'person'):
                            
#                             if (self.flag_thresh == True):
#                                 ini_thresh_frame_counter = 1
#                                 final_thresh_frame_counter = 0
#                                 self.flag_thresh = False
                            
#                             final_thresh_frame_counter += 1
#                             thresh_difference = int(final_thresh_frame_counter - ini_thresh_frame_counter)
                
#                             if (thresh_difference >= (self.thresh * self.cap_rate)):
                                
#                                 if (self.flag_patience == True):
#                                     print('[INFO] Raise the Alarm(1)')
#                                     ini_patience_frame_counter = 1
#                                     final_patience_frame_counter = 0
#                                     self.flag_patience = False
                                
#                                 final_patience_frame_counter += 1
#                                 patience_difference = int(final_patience_frame_counter - ini_patience_frame_counter)
                            
#                                 if (patience_difference >= (self.patience * self.cap_rate)):
#                                     print('[INFO] Raise the Alarm(2)...')
#                                     final_patience_frame_counter = 0
                                
#                         else: 
#                             print('Entered Else.....!!!!')
#                             self.flag_thresh = True
#                             self.flag_patience = True
                                
#                 if (len(boxes) == 0):
#                     print('New Condition Initiated')
#                     self.flag_thresh = True
#                     self.flag_patience = True
    
#                         ###################################################
                        
#                 # elapsed_time = time.time() - starting_time
#                 # fps=frame_id/elapsed_time
#                 # cv2.putText(frame,"FPS:"+str(round(fps,2)),(10,50),font,2,(0,0,0),1)
                
                cv2.imshow("Image",frame)
                
                key = cv2.waitKey(1)
                if key == 27: #esc key stops qthe process
                    break
                
            ################################
            # Frame counter reinitialization
            ################################
            
            if frame_counter == condition: 
                frame_counter = 0
                
            frame_counter = frame_counter + 1
            
            ################################
                
                
            # ################################
            # # Old Frame counter reinitialization
            # ################################
                
            # last_highest_multiple = (self.frame_rate // self.cap_rate) * self.cap_rate
            
            # if frame_counter == last_highest_multiple: 
            #     frame_counter = 0
                
            # frame_counter = frame_counter + 1
            
            # ################################
            
            print('[INFO] Out of loop...')
            
        cap.release()    
        cv2.destroyAllWindows()
        
        
if __name__ == '__main__':
    a =  CaptureRate('./yolo_files/yolov3.cfg.txt', './yolo_files/yolov3.weights', 
                                       './yolo_files/coco.names.txt', 0, patience = 2)
    a.main_run()      
        
